In [1]:
import pandas as pd 
import numpy as np 
from datetime import date
%run formulas.ipynb 

In [2]:
def combine_team_games(df, keep_method='home'):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_A', '_B'],
                      on=['GAME_ID'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        pass
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_A.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result

In [3]:
from nba_api.stats.static import teams

nba_teams = teams.get_teams()
# nba_teams

ModuleNotFoundError: No module named 'nba_api'

In [ ]:
#Getting The Year/Season
currentDT = datetime.now()
date = currentDT.date()
month = date.strftime("%m")
year = date.strftime("%Y")
seasons = []
if int(month) < 10:
    year1 = int(year) - 1
    season1 = str(year1) + "-" + year[2:4]
    seasons.append(season1)
    year2 = year1 - 1
    season2 = str(year2) + "-"+ str(year1)[2:4]
    seasons.append(season2)
else:
    year1 = int(year) + 1
    season1 = year + "-" + str(year1)[2:4]
    seasons.append(season1)
    year2 = int(year) - 1
    season2 = str(year2) + "-"+ year[2:4]
    seasons.append(season2)

In [ ]:
class Team_Cumulative_Stats:
    def __init__(self, team_abbr): # team_abbr is the abbreviated name of the team city. EX: ATL for Alanta Hawks
        self.team = team_abbr
        # The first element of the list is 0 for all features. This is because there are no values for the first game.
        self.nth_game = 0 # used in part 1 (see below)
        self.curr_game = 0 # used in part 2 (see below)

        # this class tracks the features for 1 team, so we exclude all the team_b stats
        features = [col[:-2] for col in games.columns
                    if col[-2:] != '_B' and col[-3:] != '_ID' and col != 'GAME_DATE']

        # excluded features I decided not to include
        excluded = ['TEAM_ABBREVIATION', 'TEAM_NAME', 'WL', 'TEAM_ID']

        self.features =  {feature:[np.nan]
                          for feature in features
                          if feature not in excluded}
        self.features['GAME_ID'] = []

    def add_game(self, game, home):
        '''
        game -- a row of the games dataframe
        home -- True if the team represented by this class is home, False if away
        '''
        suffix = '_A' if home else '_B' # in the games dataframe, _A is for the hometeam, and _B is for away
        for feature in self.features:
            if feature != 'GAME_ID': # all of the stats have to be averaged, except for GAME_ID

                # if this is the first game, there is nothing to average
                if self.nth_game == 0:
                    new_val = row.loc[feature + suffix]

                # otherwise, do the math to get the next average value
                else:
                    last_entry = self.features[feature][self.nth_game]
                    if last_entry == np.nan:
                        last_entry = 0
                    new_val = (last_entry * self.nth_game + row.loc[feature + suffix]) / (self.nth_game + 1)
                self.features[feature].append(new_val)
        self.features['GAME_ID'].append(row.loc['GAME_ID'])
        self.nth_game += 1

    def get_next_game(self, home): # Used to use self.features, once it is built, to create the DataFrame (part 2)
        '''home --- True if the team is home, False if away'''
        suffix = '_A' if home else '_B'
        result = {feature + suffix : [self.features[feature][self.curr_game]] for feature in self.features
                 if feature != 'GAME_ID'}
        result['GAME_ID'] = [self.features['GAME_ID'][self.curr_game]]
        self.curr_game += 1
        return result

In [ ]:
combinedDF = pd.DataFrame()

from nba_api.stats.endpoints import leaguegamefinder
for season in seasons:
    gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable = season)
    # The first DataFrame of those returned is what we want.
    games = combine_team_games(gamefinder.get_data_frames()[0], keep_method = 'home')
    games = games[games.GAME_ID.str[0:3] == '002']
    games.sort_values('GAME_ID', inplace = True)
    games.reset_index(inplace = True)
    games['WL_A'].replace({'W' : 1, 'L' : 0}, inplace = True)
    games.rename(columns = {'GAME_DATE_A' : 'GAME_DATE', 'SEASON_ID_A' : 'SEASON_ID'}, inplace = True)


    # get columns for four factors
    four_factors = {'efg_pct':{'a':[], 'b':[]},
                    'tov_pct':{'a':[], 'b':[]},
                    'orb_pct':{'a':[], 'b':[]},
                    'ft/fga':{'a':[], 'b':[]}}
    for ind, row in games.iterrows():
        for factor in four_factors:
            if factor == 'efg_pct':
                four_factors[factor]['a'].append(efg_pct(row, True))
                four_factors[factor]['b'].append(efg_pct(row, False))
            elif factor == 'tov_pct':
                four_factors[factor]['a'].append(tov_pct(row, True))
                four_factors[factor]['b'].append(tov_pct(row, False))
            elif factor == 'orb_pct':
                four_factors[factor]['a'].append(orb_pct(row, True))
                four_factors[factor]['b'].append(orb_pct(row, False))
            else:
                four_factors[factor]['a'].append(ft_per_fga(row, True))
                four_factors[factor]['b'].append(ft_per_fga(row, False))

    columns = [cols for cols in games.columns]
    loc_a = columns.index('FTA_A')
    for factor in four_factors:
        games.insert(loc=loc_a, column=factor.upper() + '_A', value=four_factors[factor]['a'])
        loc_a = columns.index('FTA_A')

    loc_b = columns.index('FTA_B')
    for factor in four_factors:
        games.insert(loc=loc_b, column=factor.upper() + '_B', value=four_factors[factor]['b'])
        loc_a = columns.index('FTA_B')

    games.drop(['FGM_A', 'FG3M_A', 'FGA_A', 'FG_PCT_A', 'FG3A_A', 'FG3_PCT_A', 'FTM_A', 'OREB_A', 'DREB_A',
                'FTA_A', 'FT_PCT_A', 'REB_A', 'AST_A', 'STL_A', 'BLK_A', 'TOV_A', 'MIN_A', 'PF_A', 'MATCHUP_A',
                'FGM_B', 'FG3M_B', 'FGA_B', 'FG_PCT_B', 'FG3A_B', 'FG3_PCT_B', 'FTM_B', 'OREB_B', 'DREB_B',
                'FTA_B', 'FT_PCT_B', 'REB_B', 'AST_B', 'STL_B', 'BLK_B', 'TOV_B', 'MIN_B', 'PF_B', 'MATCHUP_B',
                'WL_B', 'GAME_DATE_B',
                'index'], axis = 1, inplace = True)

    # columns = [cols for cols in games.columns]
    # columns
    games

    # dictionary with a teams city abbreviation as the key, and a Team_Cumulative_Stats object for that team as the value
    # see the above cell for the definition of nba_teams
    teams = {team['abbreviation']:Team_Cumulative_Stats(team['abbreviation']) for team in nba_teams}

    # part 1 (build the data from the games dataframe)
    for ind, row in games.iterrows():
        team_a = teams[row.loc['TEAM_ABBREVIATION_A']]
        team_b = teams[row.loc['TEAM_ABBREVIATION_B']]
        team_a.add_game(row, True)
        team_b.add_game(row, False)

    # part 2  (gather that data back together into a dictionary with the two competing teams on the same line)
    for ind, row in games.iterrows():
        team_a = row.loc['TEAM_ABBREVIATION_A']
        team_b = row.loc['TEAM_ABBREVIATION_B']
        a = teams[team_a].get_next_game(True)
        b = teams[team_b].get_next_game(False)
        if ind == 0: # if this is the first loop, initialize the training_data
            training_data = {**a, **b} # training data is a dictionary, which will become the dataframe after it is fully built
        else: # build the training data using values returned by the get_next_game method
            for feature in a:
                training_data[feature].append(a[feature][0])
            for feature in b:
                if feature != 'GAME_ID':
                    training_data[feature].append(b[feature][0])

    # create the dataframe, add the WL_A column, and PTS_A, PTS_B columns
    test = pd.DataFrame.from_dict(training_data)
    test['WL_A'] = games['WL_A']
    test.rename(columns = {'PTS_A' : 'AVG_PTS_A', 'PTS_B' : 'AVG_PTS_B'}, inplace = True)
    test['PTS_A'], test['PTS_B'] = games['PTS_A'], games['PTS_B']

    test.shape
    #Combine each season's dataframe into 1?
    combinedDF = pd.concat([combinedDF, test])

combinedDF.reset_index(drop=True, inplace=True)
combinedDF.style

1. Check to make sure that the dataframe I made actually represents what happened
2. Remove useless stats, add useful stats that we discussed last week
3. Make this work useful for an arbitrary # of games
4. Use the techniques in [here](https://math.stackexchange.com/questions/684519/what-is-the-most-scientific-way-to-assign-weights-to-historical-data) to weigh recent years more

In [ ]:
from nba_api.stats.endpoints import cumestatsteam
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.static import teams

nba_teams = teams.get_teams()
season = '2013-14'
season_type = 'Regular Season'
gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable = season)
games = combine_team_games(gamefinder.get_data_frames()[0], keep_method = 'home')

sixers = [team['id'] for team in nba_teams if team['abbreviation'] == 'PHI']
print(sixers)
game_ids = [] 
n = 20 
i = 0 

# get the first 20 games the sixers played in the 2013-14 season
for ind, row in games.iterrows(): 
    # if 'PHI' == row.loc['TEAM_ABBREVIATION_A'] or 'PHI' == row.loc['TEAM_ABBREVIATION_B']: 
    game_ids.append(row.loc['GAME_ID'])
    i += 1 
    if i == 20: 
        break 

df = cumestatsteam.CumeStatsTeam(league_id = '00', game_ids = game_ids, season = season, season_type_all_star = season_type, team_id = sixers[0])
df.get_data_frames()[1]

In [ ]:
from nba_api.stats.endpoints import teamdashboardbygeneralsplits

start = '10/29/2013' 
end = '11/04/2013'

generalTeamInfo = teamdashboardbygeneralsplits.TeamDashboardByGeneralSplits(team_id=sixers[0], per_mode_detailed='Per100Possessions', date_from_nullable=start, date_to_nullable=end, season=season, timeout=120)
generalTeamDict = generalTeamInfo.get_normalized_dict()
generalTeamDashboard = generalTeamDict['OverallTeamDashboard'][0]
generalTeamDashboard